In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import ast

### Filtragem dos dados do arquivo movies.csv

In [2]:
df = pd.read_csv("../../dados_brutos/movies_metadata.csv")
df.head()

C:\Users\felip\AppData\Local\Temp\ipykernel_31260\1897420239.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../dados_brutos/movies_metadata.csv")


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [3]:
df_filtered = df[df['genres'].apply(lambda x: "Comedy" in x and "Romance" in x)]
df_sort = df_filtered.sort_values(by="revenue",ascending=False).head(1000)

drop_df = ["homepage", "poster_path", "video", "imdb_id", "overview", "original_title", 
           "spoken_languages", "tagline","original_language","status","genres","spoken_languages","adult"
          ,"production_companies","production_countries","vote_count","popularity"]
df = df_sort.drop(drop_df, axis=1)


df["id"] = pd.to_numeric(df['id'], errors='coerce', downcast="integer")
df["budget"] =pd.to_numeric(df['budget'], errors='coerce', downcast="float") 
df['release_date'] = pd.to_datetime(df['release_date'])
df = df[df["budget"] > 0]
df["belongs_to_collection"] = df["belongs_to_collection"].apply(lambda x: ast.literal_eval(x) if pd.notna(x) else x)
df["belongs_to_collection"] = df["belongs_to_collection"].apply(lambda x: int(x["id"]) if pd.notna(x) else x)

df=df.rename(columns={"belongs_to_collection":"idColecao","budget":"custo","id":"idFilme","release_date":"lancamento",
                     "revenue":"bilheteria","runtime":"duracao","title":"titulo","vote_average":"nota"})
df.to_csv("../../dados_normalizados/movies_base.csv",index=False)

### Filtragem dos dados do arquivo credits.csv

In [4]:
credits = pd.read_csv("../../dados_brutos/credits.csv")
credits["crew"] = credits["crew"].apply(lambda x: ast.literal_eval(x))
credits["cast"] = credits["cast"].apply(lambda x: ast.literal_eval(x))
test = credits[credits["id"].isin(df["idFilme"])]
diretores = test.loc[:,["crew","id"]]
diretores["crew"] = diretores["crew"].apply(lambda x: [i for i in x if i["job"] == "Director"])

### Cria os arquivos de dados dos atores e diretores

In [5]:
data = []
for i in diretores["crew"]:
    for j in i:
        data.append([j["id"],j["name"],j["gender"]])
dire = pd.DataFrame(data, columns =["id","name","gender"])
dire = dire.drop_duplicates(subset="id")

atores = test.loc[:,["cast","id"]]
data = []
for i in atores["cast"]:
    for j in i:
        if "(Voice)" in j["name"]:
            j["name"].replace("(Voice)","")
        
        data.append([j["id"],j["name"],j["gender"]])
        
at = pd.DataFrame(data,columns =["id","name","gender"])
at = at.drop_duplicates(subset="id")
dire.to_csv("../../dados_normalizados/diretores.csv",index=False)
at.to_csv("../../dados_normalizados/atores.csv",index=False)

### Criação da Tabela Coleção 

In [6]:
df = pd.read_csv("../../dados_brutos/movies_metadata.csv")
f = pd.DataFrame(df["belongs_to_collection"])
f = f.dropna()
f["belongs_to_collection"] = f["belongs_to_collection"].apply(lambda x:ast.literal_eval(x))
li = []
for i in f["belongs_to_collection"]:
     if isinstance(i, dict):
        li.append({"id": i.get("id"), "nome": i.get("name")})
collection = pd.DataFrame(li)
collection=collection.drop_duplicates(keep="first")
collection.to_csv("../../dados_normalizados/collections.csv")

C:\Users\felip\AppData\Local\Temp\ipykernel_31260\1612385736.py:1: DtypeWarning: Columns (10) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("../../dados_brutos/movies_metadata.csv")


### Criação de uma tabela para elencar cada diretor a mais de um filme

In [7]:
data = []
for _,i in atores.iterrows():
    id = i["id"]
    for j in i["cast"]:
        data.append([j["id"],id])
elenco = pd.DataFrame(data, columns=["idAtor","idFilme"])
elenco.to_csv("../../dados_normalizados/elenco.csv")

### Criação de uma tabela para elencar cada ator a mais de um filme

In [8]:
data = []
for _,i in diretores.iterrows():
    id = i["id"]
    for j in i["crew"]:
        data.append([j["id"],id])
direcao = pd.DataFrame(data, columns=["idDiretor","idFilme"])
direcao.to_csv("../../dados_normalizados/direcao.csv")